Data Collection:

The dataset is available in UC Irvine Machine Learning Repository. We followed the import in Python instructions on the website to get the dataset.

In [25]:
pip install ucimlrepo

Note: you may need to restart the kernel to use updated packages.


In [26]:
from scipy.stats import skew
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from ucimlrepo import fetch_ucirepo 
import pandas as pd
import os

In [73]:

# fetch dataset 
bank_marketing = fetch_ucirepo(id=222) 
  
# data (as pandas dataframes) 
X = bank_marketing.data.features 
y = bank_marketing.data.targets 
  
# metadata 
print(bank_marketing.metadata) 
  
# variable information 
print(bank_marketing.variables) 

{'uci_id': 222, 'name': 'Bank Marketing', 'repository_url': 'https://archive.ics.uci.edu/dataset/222/bank+marketing', 'data_url': 'https://archive.ics.uci.edu/static/public/222/data.csv', 'abstract': 'The data is related with direct marketing campaigns (phone calls) of a Portuguese banking institution. The classification goal is to predict if the client will subscribe a term deposit (variable y).', 'area': 'Business', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 45211, 'num_features': 16, 'feature_types': ['Categorical', 'Integer'], 'demographics': ['Age', 'Occupation', 'Marital Status', 'Education Level'], 'target_col': ['y'], 'index_col': None, 'has_missing_values': 'yes', 'missing_values_symbol': 'NaN', 'year_of_dataset_creation': 2014, 'last_updated': 'Fri Aug 18 2023', 'dataset_doi': '10.24432/C5K306', 'creators': ['S. Moro', 'P. Rita', 'P. Cortez'], 'intro_paper': {'ID': 277, 'type': 'NATIVE', 'title': 'A data-driven approach to predict the s

Data Processing

In [74]:
# 1. Drop the 'y' column from X to avoid duplication
if 'y' in X.columns:
    X = X.drop(columns='y')

# 2. Concatenate X and y safely
X = pd.concat([X, y], axis=1)

# 3. Drop rows where y is NaN
X = X.dropna(subset=['y'])

# 4. (Optional) Check your data
print(X.head())
print(X['y'].value_counts())


   age           job  marital  education default  balance housing loan  \
0   58    management  married   tertiary      no     2143     yes   no   
1   44    technician   single  secondary      no       29     yes   no   
2   33  entrepreneur  married  secondary      no        2     yes  yes   
3   47   blue-collar  married        NaN      no     1506     yes   no   
4   33           NaN   single        NaN      no        1      no   no   

  contact  day_of_week month  duration  campaign  pdays  previous poutcome   y  
0     NaN            5   may       261         1     -1         0      NaN  no  
1     NaN            5   may       151         1     -1         0      NaN  no  
2     NaN            5   may        76         1     -1         0      NaN  no  
3     NaN            5   may        92         1     -1         0      NaN  no  
4     NaN            5   may       198         1     -1         0      NaN  no  
y
no     39922
yes     5289
Name: count, dtype: int64


In [75]:
#To know the data types and missing data points 
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   age          45211 non-null  int64 
 1   job          44923 non-null  object
 2   marital      45211 non-null  object
 3   education    43354 non-null  object
 4   default      45211 non-null  object
 5   balance      45211 non-null  int64 
 6   housing      45211 non-null  object
 7   loan         45211 non-null  object
 8   contact      32191 non-null  object
 9   day_of_week  45211 non-null  int64 
 10  month        45211 non-null  object
 11  duration     45211 non-null  int64 
 12  campaign     45211 non-null  int64 
 13  pdays        45211 non-null  int64 
 14  previous     45211 non-null  int64 
 15  poutcome     8252 non-null   object
 16  y            45211 non-null  object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB


In [76]:
#descriptive statistics
X.describe()

,age,balance,day_of_week,duration,campaign,pdays,previous
count,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000
mean,40.936210,1362.272058,15.806419,258.163080,2.763841,40.197828,0.580323
std,10.618762,3044.765829,8.322476,257.527812,3.098021,100.128746,2.303441
min,18.000000,-8019.000000,1.000000,0.000000,1.000000,-1.000000,0.000000
25%,33.000000,72.000000,8.000000,103.000000,1.000000,-1.000000,0.000000
50%,39.000000,448.000000,16.000000,180.000000,2.000000,-1.000000,0.000000
75%,48.000000,1428.000000,21.000000,319.000000,3.000000,-1.000000,0.000000
max,95.000000,102127.000000,31.000000,4918.000000,63.000000,871.000000,275.000000


In [77]:
#droping duplicates if they exist
X.drop_duplicates(keep='first',inplace=True)
X.shape

(45211, 17)

In [78]:
#Handling Null Values: Part 1
X_null=X.isna().sum().to_frame('null_count').reset_index()
X_null=X_null[X_null['null_count']>0].reset_index(drop=True)

X_null['null_count']=X_null['null_count']*100/X.shape[0]
X_null

,index,null_count
0,job,0.637013
1,education,4.107407
2,contact,28.798301
3,poutcome,81.747805


In [79]:
#Handling Null Values: Part 2, dropping columns with more than 80% null_values. The threshold can be set based on the dataset.
columns_to_drop = X_null[X_null['null_count'] > 80]['index'].tolist()
X.drop(columns=columns_to_drop,inplace=True)
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   age          45211 non-null  int64 
 1   job          44923 non-null  object
 2   marital      45211 non-null  object
 3   education    43354 non-null  object
 4   default      45211 non-null  object
 5   balance      45211 non-null  int64 
 6   housing      45211 non-null  object
 7   loan         45211 non-null  object
 8   contact      32191 non-null  object
 9   day_of_week  45211 non-null  int64 
 10  month        45211 non-null  object
 11  duration     45211 non-null  int64 
 12  campaign     45211 non-null  int64 
 13  pdays        45211 non-null  int64 
 14  previous     45211 non-null  int64 
 15  y            45211 non-null  object
dtypes: int64(7), object(9)
memory usage: 5.5+ MB


In [80]:
#Handling Null Values: Part 3, we are goimng to impute columns with <80% missing values

# Step 1: Identify columns with <80% missing values
impute = X_null[X_null['null_count'] < 80]['index'].tolist()

# Step 2: Iterate through each column and impute missing values
for i in impute:
    # Check if the column is categorical (dtype is 'object')
    if X[i].dtype == 'object':
        # Fill missing values with the most frequent value (mode)
        X[i].fillna(X[i].mode()[0], inplace=True)
    else:
        # Fill missing values in numerical columns with median (immune to outliers)
        X[i].fillna(X[i].median(), inplace=True)

print("Missing values imputed successfully!")
print(X.columns)

Missing values imputed successfully!
Index(['age', 'job', 'marital', 'education', 'default', 'balance', 'housing',
       'loan', 'contact', 'day_of_week', 'month', 'duration', 'campaign',
       'pdays', 'previous', 'y'],
      dtype='object')


In [81]:
# All the null values are handled
X.isna().sum()
X['y']

0         no
1         no
2         no
3         no
4         no
        ... 
45206    yes
45207    yes
45208    yes
45209     no
45210     no
Name: y, Length: 45211, dtype: object

In [82]:
#Partitioning numerical and categorical values
X_numerical=X[X.describe().columns]
X_categorical=X[[i for i in X.columns if i not in X_numerical.columns ]]

In [83]:
# We are encoding categorical values 

# 1) Month- we are doing ordinal encoding 

# Define the ordinal mapping
month_order = {'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6, 
               'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12}

# Apply ordinal encoding
X['month_encoded'] = X['month'].map(month_order)

print(X[['month', 'month_encoded']].head())
print(X.head)

  month  month_encoded
0   may              5
1   may              5
2   may              5
3   may              5
4   may              5
<bound method NDFrame.head of        age           job   marital  education default  balance housing loan  \
0       58    management   married   tertiary      no     2143     yes   no   
1       44    technician    single  secondary      no       29     yes   no   
2       33  entrepreneur   married  secondary      no        2     yes  yes   
3       47   blue-collar   married  secondary      no     1506     yes   no   
4       33   blue-collar    single  secondary      no        1      no   no   
...    ...           ...       ...        ...     ...      ...     ...  ...   
45206   51    technician   married   tertiary      no      825      no   no   
45207   71       retired  divorced    primary      no     1729      no   no   
45208   72       retired   married  secondary      no     5715      no   no   
45209   57   blue-collar   married  second

In [84]:
# 2- job, maritial, contactwe are currently going with label encoding since it's good for tree based models may change in the future

from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder for 'job'
le_job = LabelEncoder()
X['job_encoded'] = le_job.fit_transform(X['job'])

print(X[['job', 'job_encoded']].head())

# Initialize LabelEncoder for 'marital'
le_marital = LabelEncoder()
X['marital_encoded'] = le_marital.fit_transform(X['marital'])

print(X[['marital', 'marital_encoded']].head())

# Initialize LabelEncoder
le_contact = LabelEncoder()

# Apply encoding
X['contact_encoded'] = le_contact.fit_transform(X['contact'])

# Display results
print(X[['contact', 'contact_encoded']].head())


            job  job_encoded
0    management            4
1    technician            9
2  entrepreneur            2
3   blue-collar            1
4   blue-collar            1
   marital  marital_encoded
0  married                1
1   single                2
2  married                1
3  married                1
4   single                2
    contact  contact_encoded
0  cellular                0
1  cellular                0
2  cellular                0
3  cellular                0
4  cellular                0


In [85]:
#3- education ordinal encoding
from sklearn.preprocessing import OrdinalEncoder

# Define the order of education levels
education_order = [['primary', 'secondary', 'tertiary']]

# Initialize OrdinalEncoder
ordinal_encoder = OrdinalEncoder(categories=education_order)

# Apply encoding
X['education_encoded'] = ordinal_encoder.fit_transform(X[['education']])

print(X[['education', 'education_encoded']].head())

   education  education_encoded
0   tertiary                2.0
1  secondary                1.0
2  secondary                1.0
3  secondary                1.0
4  secondary                1.0


In [86]:
X['loan'].value_counts()


loan
no     37967
yes     7244
Name: count, dtype: int64

In [87]:
#4- Binary columns yes/ no- binary encoding

# Define mapping for Yes/No columns
binary_mapping = {'yes': 1, 'no': 0}

# Apply encoding
X['default_encoded'] = X['default'].map(binary_mapping)
X['housing_encoded'] = X['housing'].map(binary_mapping)
X['loan_encoded'] = X['loan'].map(binary_mapping)
X['y_encoded']=X['y'].map(binary_mapping)
# Display results
print(X[['default', 'default_encoded', 'housing', 'housing_encoded', 'loan', 'loan_encoded','y','y_encoded']].head())





  default  default_encoded housing  housing_encoded loan  loan_encoded   y  \
0      no                0     yes                1   no             0  no   
1      no                0     yes                1   no             0  no   
2      no                0     yes                1  yes             1  no   
3      no                0     yes                1   no             0  no   
4      no                0      no                0   no             0  no   

   y_encoded  
0          0  
1          0  
2          0  
3          0  
4          0  


In [88]:
# Drop original categorical columns as they have encoded versions
categorical_columns = ["job", "marital", "education", "default", "housing", "loan", "contact", "month", "y"]
X = X.drop(columns=categorical_columns, errors="ignore")

# Display the dataframe after dropping redundant columns
print("Columns after dropping redundant categorical columns:")
print(X.head())
print(X.columns)

Columns after dropping redundant categorical columns:
   age  balance  day_of_week  duration  campaign  pdays  previous  \
0   58     2143            5       261         1     -1         0   
1   44       29            5       151         1     -1         0   
2   33        2            5        76         1     -1         0   
3   47     1506            5        92         1     -1         0   
4   33        1            5       198         1     -1         0   

   month_encoded  job_encoded  marital_encoded  contact_encoded  \
0              5            4                1                0   
1              5            9                2                0   
2              5            2                1                0   
3              5            1                1                0   
4              5            1                2                0   

   education_encoded  default_encoded  housing_encoded  loan_encoded  \
0                2.0                0                1  

In [89]:
import numpy as np
from scipy.stats import skew
from sklearn.preprocessing import StandardScaler

# Identify numerical columns (excluding encoded categorical features)
numerical_cols = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
numerical_cols = [col for col in numerical_cols if not col.endswith("_encoded")]

# Identify binary numerical columns (0/1 values but NOT `_encoded`)
binary_cols = [col for col in numerical_cols if set(X[col].dropna().unique()).issubset({0, 1})]

# Separate non-binary numerical columns
non_binary_numerical_cols = [col for col in numerical_cols if col not in binary_cols]

# Check for skewness in non-binary numerical columns
skewed_cols = X[non_binary_numerical_cols].apply(lambda x: skew(x.dropna()))

# Apply log transformation to highly skewed non-binary numerical columns (absolute skewness > 1)
for col in skewed_cols[abs(skewed_cols) > 1].index:
    if (X[col] <= 0).any():  # If there are zero or negative values, shift before log transformation
        X[col] = np.log1p(X[col] - X[col].min() + 1)
    else:
        X[col] = np.log1p(X[col])

# Standardize only non-binary numerical features
scaler = StandardScaler()
X[non_binary_numerical_cols] = scaler.fit_transform(X[non_binary_numerical_cols])

# Ensure binary columns are untouched (0/1)
for col in binary_cols:
    X[col] = X[col].astype(int)  # Convert back to ensure no changes

# Display summary statistics
print("Final dataset after normalization and standardization:")
print(X.describe())

# Final check
print("Final dataset shape:", X.shape)
print("Final dataset info:")
print(X.columns)


Final dataset after normalization and standardization:
                age       balance   day_of_week      duration      campaign  \
count  4.521100e+04  4.521100e+04  4.521100e+04  4.521100e+04  4.521100e+04   
mean   2.112250e-16 -2.942063e-15  1.257292e-17  7.065980e-16 -3.520417e-17   
std    1.000011e+00  1.000011e+00  1.000011e+00  1.000011e+00  1.000011e+00   
min   -2.159994e+00 -3.968321e+01 -1.779108e+00 -4.928214e+00 -9.094571e-01   
25%   -7.473845e-01 -5.652955e-01 -9.380027e-01 -5.788669e-01 -9.094571e-01   
50%   -1.823406e-01 -3.514087e-01  2.326031e-02  2.513596e-02 -1.171495e-01   
75%    6.652252e-01  1.643000e-01  6.240497e-01  6.482326e-01  4.450018e-01   
max    5.091402e+00  1.173122e+01  1.825628e+00  3.645616e+00  5.862837e+00   

              pdays      previous  month_encoded   job_encoded  \
count  4.521100e+04  4.521100e+04   45211.000000  45211.000000   
mean   4.023334e-17 -1.408167e-16       6.144655      4.276061   
std    1.000011e+00  1.000011e+00  

In [92]:
import os

# Get the current working directory
current_dir = os.getcwd()

# Move one level up from the current directory
parent_dir = os.path.abspath(os.path.join(current_dir, ".."))

# Define the destination path dynamically
save_path = os.path.join(parent_dir, "data", "Data Preprocessing", "data_processing.csv")

X=X.rename(columns={'y_encoded':'y'})

# Save X as a CSV file
X.to_csv(save_path, index=False)

print(f"Dataset saved successfully at: {save_path}")


Dataset saved successfully at: d:\NEU\Spring 2025\Data_Mining\Predictive-Modeling-for-Bank-Term-Deposit-Subscriptions\data\Data Preprocessing\data_processing.csv


## **Final Dataset Summary**

- The dataset is now **clean, structured, and fully prepared for model training**.  
- **All missing values are handled**, and categorical variables are properly encoded.  
- **Redundant duplicate columns have been removed**, ensuring only necessary features are retained.  
- **Numerical features have been standardized/normalized**, improving model performance and ensuring consistency in scale.  
- The dataset is now **optimized and ready to be used for predictive modeling**. 🚀  


In [79]:
X['default_encoded'].value_counts()

default_encoded
0    44396
1      815
Name: count, dtype: int64